# 구글드라이브 연결

In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "gdrive/MyDrive/추천시스템/teamproject/"
os.listdir(path)

Mounted at /content/gdrive


['links.csv',
 'tags.csv',
 'movies.csv',
 'ratings.csv',
 '협업 필터링 - User-based with dot production.ipynb',
 'ALS 제품 추천 시스템.ipynb',
 'CF- Item-based_dot product_ASL + normal.ipynb',
 '무비렌즈 데이터 전처리.ipynb']

# 데이터 로드

https://grouplens.org/datasets/movielens/

https://files.grouplens.org/datasets/movielens/ml-10m-README.html

In [2]:
import pandas as pd
import numpy as np

In [209]:
movies = pd.read_csv(path + "movies.csv")
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [210]:
ratings = pd.read_csv(path + "ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [211]:
tags = pd.read_csv(path + "tags.csv")
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


# 영화 데이터

## 중복체크

In [212]:
len(movies)  # 무비 데이터 수

9742

In [213]:
movies['movieId'].nunique()  # 무비 id 수

9742

In [214]:
movies['title'].nunique()  # 무비 제목 != 무비 id 수

9737

In [215]:
movies[movies.duplicated('title')]

,movieId,title,genres
5601,26958,Emma (1996),Romance
6932,64997,War of the Worlds (2005),Action|Sci-Fi
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9135,147002,Eros (2004),Drama|Romance
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [216]:
movies['title'].value_counts()[movies['title'].value_counts() > 1] #value count 를 1개 이상

Emma (1996)                               2
War of the Worlds (2005)                  2
Confessions of a Dangerous Mind (2002)    2
Eros (2004)                               2
Saturn 3 (1980)                           2
Name: title, dtype: int64

In [217]:
duplicate_count1 = movies.duplicated().sum()
print("movies 중복된 행의 개수:", duplicate_count1)
duplicate_count2 = ratings.duplicated().sum()
print("ratings 중복된 행의 개수:", duplicate_count2)
duplicate_count3 = tags.duplicated().sum()
print("tags 중복된 행의 개수:", duplicate_count3)

movies 중복된 행의 개수: 0
ratings 중복된 행의 개수: 0
tags 중복된 행의 개수: 0


## 중복값 처리

#### 중복값 데이터 처리 방법

영화1) Emma

movies 데이터 처리

In [218]:
movies[movies['title'] == 'Emma (1996)']

,movieId,title,genres
650,838,Emma (1996),Comedy|Drama|Romance
5601,26958,Emma (1996),Romance


In [219]:
movies = movies[~(movies['movieId'] == 26958)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


ratings 데이터 처리

In [220]:
ratings[ratings['movieId'] == 26958] # 기존의 데이터에서 제거했기에 일치하게 하기 위해 제거

,userId,movieId,rating,timestamp
80596,509,26958,3.5,1436031753


In [221]:
ratings.loc[80596, 'movieId'] = 838

In [222]:
ratings[ratings['movieId'] == 26958]

,userId,movieId,rating,timestamp


In [223]:
ratings.loc[[80596]]

,userId,movieId,rating,timestamp
80596,509,838,3.5,1436031753


tags 데이터 처리

In [224]:
tags[tags['movieId'] == 26958] # 기존의 데이터에서 제거했기에 일치하게 하기 위해 제거

,userId,movieId,tag,timestamp


영화2) War of the Worlds

movies 데이터 처리

In [225]:
movies[movies['title'] == 'War of the Worlds (2005)']

,movieId,title,genres
5931,34048,War of the Worlds (2005),Action|Adventure|Sci-Fi|Thriller
6932,64997,War of the Worlds (2005),Action|Sci-Fi


In [226]:
movies = movies[~(movies['movieId'] == 64997)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


ratings 데이터 처리

In [227]:
ratings[ratings['movieId'] == 64997]

,userId,movieId,rating,timestamp
4747,28,64997,3.5,1234850075
11451,68,64997,2.5,1230497715


In [228]:
ratings.loc[4747, 'movieId'] = 34048
ratings.loc[11451, 'movieId'] = 34048

In [229]:
ratings[ratings['movieId'] == 64997]

,userId,movieId,rating,timestamp


In [230]:
ratings.loc[[4747, 11451]]

,userId,movieId,rating,timestamp
4747,28,34048,3.5,1234850075
11451,68,34048,2.5,1230497715


tags 데이터 처리

In [231]:
tags[tags['movieId'] == 64997] # 기존의 데이터에서 제거했기에 일치하게 하기 위해 제거

,userId,movieId,tag,timestamp


### 위의 과정을 tag.csv 파일에서도 똑같이 진행해야된다

영화3) Confessions of a Dangerous Mind

movies 데이터 처리 1st

In [232]:
movies[movies['title'] == 'Confessions of a Dangerous Mind (2002)']

,movieId,title,genres
4169,6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Thriller
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller


In [233]:
movies.loc[4169,'genres'] = 'Comedy|Crime|Drama|Romance|Thriller'
movies[movies['title'] == 'Confessions of a Dangerous Mind (2002)']

,movieId,title,genres
4169,6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller


In [234]:
movies = movies[~(movies['movieId'] == 144606)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [235]:
ratings[ratings['movieId'] == 144606]

,userId,movieId,rating,timestamp
17819,111,144606,4.0,1517441257


In [236]:
ratings.loc[17819, 'movieId'] = 6003

In [237]:
ratings[ratings['movieId'] == 144606] # 144606이 있는지 확인

,userId,movieId,rating,timestamp


In [238]:
ratings.loc[[17819]]

,userId,movieId,rating,timestamp
17819,111,6003,4.0,1517441257


tags 중복값 제거

In [239]:
tags[tags['movieId'] == 144606]

,userId,movieId,tag,timestamp


영화4) Eros

movie 데이터 처리

In [240]:
movies[movies['title'] == 'Eros (2004)']

,movieId,title,genres
5854,32600,Eros (2004),Drama
9135,147002,Eros (2004),Drama|Romance


In [241]:
movies = movies[~(movies['movieId'] == 32600)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


ratings 데이터 처리

In [242]:
ratings[ratings['movieId'] == 32600]

,userId,movieId,rating,timestamp
98357,606,32600,3.5,1171410642


In [243]:
ratings.loc[98357, 'movieId'] = 147002

In [244]:
ratings[ratings['movieId'] == 32600]

,userId,movieId,rating,timestamp


In [245]:
ratings.loc[[98357]] # 수정 정보 확인

,userId,movieId,rating,timestamp
98357,606,147002,3.5,1171410642


In [246]:
tags[tags['movieId'] == 32600]

,userId,movieId,tag,timestamp


영화5) Saturn 3

In [247]:
movies[movies['title'] == 'Saturn 3 (1980)']

,movieId,title,genres
2141,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [248]:
movies = movies[~(movies['movieId'] == 168358)].copy()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


ratings 데이터 처리

In [249]:
ratings[ratings['movieId'] == 168358]

,userId,movieId,rating,timestamp
81458,514,168358,2.5,1533945970


In [250]:
ratings.loc[81458, 'movieId'] = 2851

In [251]:
ratings[ratings['movieId'] == 168358]

,userId,movieId,rating,timestamp


In [252]:
ratings.loc[[81458]]

,userId,movieId,rating,timestamp
81458,514,2851,2.5,1533945970


tags 데이터 처리

In [253]:
tags[tags['movieId'] == 168358]

,userId,movieId,tag,timestamp


처리 확인

In [254]:
len(movies)

9737

In [255]:
movies['movieId'].nunique()

9737

In [256]:
movies['title'].nunique()

9737

## 장르 결측치 처리

Imputation

일일이 확인해서 결측치를 채워주는 것이 Drop보다 정확도를 올릴 수 있다

index 번호 초기화

In [257]:
movies = movies.reset_index(drop=True)
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9732,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9733,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9734,193585,Flint (2017),Drama
9735,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [258]:
ratings = ratings.reset_index(drop=True)
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [259]:
tags = tags.reset_index(drop=True)
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [260]:
no_genre_cond = movies['genres'].str.contains(' ') # 공백이 있는지 확인
no_genere = movies[no_genre_cond]

In [261]:
no_genere

,movieId,title,genres
8514,114335,La cravate (1957),(no genres listed)
8681,122888,Ben-hur (2016),(no genres listed)
8684,122896,Pirates of the Caribbean: Dead Men Tell No Tal...,(no genres listed)
8779,129250,Superfast! (2015),(no genres listed)
8833,132084,Let It Be Me (1995),(no genres listed)
8899,134861,Trevor Noah: African American (2013),(no genres listed)
9030,141131,Guardians (2016),(no genres listed)
9050,141866,Green Room (2015),(no genres listed)
9067,142456,The Brand New Testament (2015),(no genres listed)
9088,143410,Hyena Road,(no genres listed)


In [262]:
ts_movies = movies.copy()

In [263]:
no_genere_title = no_genere['title']

In [288]:
len(no_genere_title)

34

In [265]:
selected_row = ts_movies.iloc[8514]
selected_row

movieId                114335
title       La cravate (1957)
genres     (no genres listed)
Name: 8514, dtype: object

In [266]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 불용어를 english로 지정하고 tf-idf 계산
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(ts_movies['title'])

# Cosine 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim = pd.DataFrame(cosine_sim, index=ts_movies.index, columns=ts_movies.index)

# index-title을 뒤집는다
indices = pd.Series(ts_movies.index, index=ts_movies['title'])

# 영화제목을 받아서 추천 영화를 돌려주는 함수
def content_recommender(title, n_of_recomm):
    # title에서 영화 index 받아오기
    idx = indices[title]
    # 주어진 영화와 다른 영화의 similarity를 가져온다
    sim_scores = cosine_sim[idx]
    # similarity 기준으로 정렬하고 n_of_recomm만큼 가져오기 (자기자신은 빼기)
    sim_scores = sim_scores.sort_values(ascending=False)[1:n_of_recomm+1]
    # 영화 title 반환
    return ts_movies.loc[sim_scores.index]['title']

# # 추천받기
# print(content_recommender("A Midsummer Night's Dream (2016)", 5))

In [267]:
movie_indices = []
for movie in no_genere_title:
    movie_idx = ts_movies[ts_movies['title'] == movie].index
    if len(movie_idx) > 0:
        movie_indices.append(movie_idx[0])

print(movie_indices)


[8514, 8681, 8684, 8779, 8833, 8899, 9030, 9050, 9067, 9088, 9134, 9174, 9213, 9244, 9255, 9303, 9312, 9344, 9409, 9422, 9444, 9473, 9509, 9510, 9513, 9520, 9529, 9536, 9557, 9568, 9606, 9656, 9658, 9664]


In [268]:
selected_row = ts_movies.iloc[8779]
selected_row

movieId                129250
title       Superfast! (2015)
genres     (no genres listed)
Name: 8779, dtype: object

In [340]:
def similar_title(movie_idx):
    # 주어진 영화 인덱스에 해당하는 영화 제목
    movie_title = ts_movies.loc[movie_idx, 'title']

    # 'movie_title' 영화의 유사한 영화 가져오기
    similar_movies = content_recommender(movie_title, 1)

    # print(f"영화 '{movie_title}'와 유사한 영화 인덱스:")
    # print(similar_movies)

    similar_movie_indices = []  # 유사한 영화의 인덱스를 저장할 리스트
    similar_movie_genres  = []
    # similar_movies의 인덱스 번호와 해당 영화 제목 출력
    for idx, similar_movie_title in enumerate(similar_movies):
        similar_movie_idx = ts_movies[ts_movies['title'] == similar_movie_title].index
        if len(similar_movie_idx) > 0:
            similar_movie_indices.append(similar_movie_idx[0])

    for similar_movie_idx in similar_movie_indices:
        similar_movie_genre = ts_movies.loc[similar_movie_idx, 'genres']
        similar_movie_genres.append(similar_movie_genre)

    return similar_movie_indices, similar_movie_genres

# DataFrame 복사
update_movie = ts_movies.copy()

# movie_indices 리스트의 영화들에 대해 장르 추가
for movie_idx in movie_indices:
    similar_movie_indices, similar_movie_genres = similar_title(movie_idx)
    update_movie.at[movie_idx, 'genres'] = similar_movie_genres

# 업데이트된 DataFrame 출력
# print(update_movie)


In [341]:
update_movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9732,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9733,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9734,193585,Flint (2017),Drama
9735,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [342]:
no_genre_cond = update_movie['genres'].str.contains(' ') # 공백이 있는지 확인
print(no_genre_cond)
# no_genere = update_movie[no_genre_cond]

0       False
1       False
2       False
3       False
4       False
        ...  
9732    False
9733    False
9734    False
9735    False
9736    False
Name: genres, Length: 9737, dtype: object


In [345]:
no_genere

,movieId,title,genres
8514,114335,La cravate (1957),(no genres listed)
8681,122888,Ben-hur (2016),(no genres listed)
8684,122896,Pirates of the Caribbean: Dead Men Tell No Tal...,(no genres listed)
8779,129250,Superfast! (2015),(no genres listed)
8833,132084,Let It Be Me (1995),(no genres listed)
8899,134861,Trevor Noah: African American (2013),(no genres listed)
9030,141131,Guardians (2016),(no genres listed)
9050,141866,Green Room (2015),(no genres listed)
9067,142456,The Brand New Testament (2015),(no genres listed)
9088,143410,Hyena Road,(no genres listed)


In [347]:
update_movie.iloc[9536]

movieId                                     172591
title      The Godfather Trilogy: 1972-1990 (1992)
genres                               [Crime|Drama]
Name: 9536, dtype: object

## 그외 결측치 체크

중간중간이 사라져 있어 인덱스 번호가 달라 이러한 아래의 에러가 나타나느데 이를 해결하기 위해 reset 인덱스을 반드시 해줘야 한다

In [334]:
update_movie.columns[update_movie.isna().any()].tolist() #isna NULL 값을 찾기, any()를 사용해서 모든 컬럼

[]

## 인덱스 정리

In [335]:
update_movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9732,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9733,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9734,193585,Flint (2017),Drama
9735,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


reset index을 해줘야 번호가 꼬이지 않는다 drop = True를 해줘야 버려준다

In [336]:
update_movie = update_movie.reset_index(drop=True)
update_movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9732,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9733,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9734,193585,Flint (2017),Drama
9735,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [353]:
# update_movie.iloc[9088]
update_movie.iloc[1]

movieId                             2
title                  Jumanji (1995)
genres     Adventure|Children|Fantasy
Name: 1, dtype: object

In [313]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [314]:
ratings = ratings.reset_index(drop=True)
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [315]:
tags = tags.reset_index(drop=True)
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


## 구분자 변경

TfidfVectorizer 사용하려면 space 를 seperator 로 써야

한 번에 시작하기 전에 spacebar 없는지 확인

In [316]:
# 장르 이름에는 빈칸 없다
# 만약 빈칸 있으면 _ 붙이는 처리 필요
# 예) k drama|fun --> k drama fun  (X)
#                 --> k_drama fun  (O)
update_movie[update_movie['genres'].str.contains(' ')]

ValueError: ignored

In [317]:
update_movie['genres'] = update_movie['genres'].str.replace('|',' ', regex=True)
update_movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9732,193581,Black Butler: Book of the Atlantic (2017),Action Animation Comedy Fantasy
9733,193583,No Game No Life: Zero (2017),Animation Comedy Fantasy
9734,193585,Flint (2017),Drama
9735,193587,Bungo Stray Dogs: Dead Apple (2018),Action Animation


## 연도 결측치 처리

In [318]:
#movies[~movies['title'].str.contains('(')] ## 특수문자 처리를 위해서는 \ <-- 이게 필요

In [319]:
# 제목에 연도 없는 데이터 검색
# movies[~movies['title'].str.contains('(')]  <-- \( 사용해야함
update_movie[~update_movie['title'].str.contains('\(')]

,movieId,title,genres
6057,40697,Babylon 5,Sci-Fi
9028,140956,Ready Player One,Action Sci-Fi Thriller
9088,143410,Hyena Road,NaN
9134,147250,The Adventures of Sherlock Holmes and Doctor W...,NaN
9175,149334,Nocturnal Animals,Drama Thriller
9255,156605,Paterson,NaN
9363,162414,Moonlight,Drama
9444,167570,The OA,NaN
9509,171495,Cosmos,NaN
9510,171631,Maria Bamford: Old Baby,NaN


In [320]:
update_movie.iloc[9088]

movieId        143410
title      Hyena Road
genres            NaN
Name: 9088, dtype: object

In [ ]:
# 'Babylon 5': 1994
# 'Ready Player One': 2018
# 'Nocturnal Animals': 2017
# 'Moonlight': 2009

In [ ]:
tmp = movies.copy()

In [ ]:
tmp.loc[6058, 'title'] = tmp.loc[6058, 'title'] + " (1994)"
tmp.loc[[6058]]

,movieId,title,genres
6058,40697,Babylon 5 (1994),Sci-Fi


In [ ]:
# 중복 실행 시 연도 중첩 방지 필요
tmp.loc[6058, 'title'] = tmp.loc[6058, 'title'] + " (1994)"
tmp.loc[[6058]]

,movieId,title,genres
6058,40697,Babylon 5 (1994) (1994),Sci-Fi


In [ ]:
# 연도 없는지 체크
'(' not in tmp.loc[6058, 'title']

False

연도가 없을 때만 사용하기위해 조건문 사용

In [ ]:
tmp = movies.copy()
idx = 6058
year = 1994

if '(' not in tmp.loc[idx, 'title']:
    tmp.loc[idx, 'title'] = tmp.loc[idx, 'title'] + f" ({year})"

tmp.loc[[idx]]

,movieId,title,genres
6058,40697,Babylon 5 (1994),Sci-Fi


In [ ]:
# 직접 입력
idx = 6058
year = 1994
if '(' not in movies.loc[idx, 'title']:
    movies.loc[idx, 'title'] = movies.loc[idx, 'title'] + f" ({year})"
movies.loc[[idx]]

,movieId,title,genres
6058,40697,Babylon 5 (1994),Sci-Fi


In [ ]:
# 직접 입력
idx = 9023
year = 2018
if '(' not in movies.loc[idx, 'title']:
    movies.loc[idx, 'title'] = movies.loc[idx, 'title'] + f" ({year})"
movies.loc[[idx]]

,movieId,title,genres
9023,140956,Ready Player One (2018),Action Sci-Fi Thriller


In [ ]:
# 직접 입력
idx = 9163
year = 2017
if '(' not in movies.loc[idx, 'title']:
    movies.loc[idx, 'title'] = movies.loc[idx, 'title'] + f" ({year})"
movies.loc[[idx]]

,movieId,title,genres
9163,149334,Nocturnal Animals (2017),Drama Thriller


In [ ]:
# 직접 입력
idx = 9345
year = 2009
if '(' not in movies.loc[idx, 'title']:
    movies.loc[idx, 'title'] = movies.loc[idx, 'title'] + f" ({year})"
movies.loc[[idx]]

,movieId,title,genres
9345,162414,Moonlight (2009),Drama


In [ ]:
# 제목에 연도 없는 데이터 검색
movies[~movies['title'].str.contains('\(')]

,movieId,title,genres


## 영화명/연도 분리

### 연습

split -> 왼쪽 부터 <br>
rsplit -> 오른쪽

In [ ]:
"Waiting to Exhale (1995)".rsplit(' ', 1)

['Waiting to Exhale', '(1995)']

In [ ]:
# 여백 주의! split에 영향줌
"Waiting to Exhale (1995) ".rsplit(' ', 1)

['Waiting to Exhale (1995)', '']

strip() -> 데이터 앞 뒤의 공백 제거

In [ ]:
"Waiting to Exhale (1995) ".strip()

'Waiting to Exhale (1995)'

In [ ]:
"Waiting to Exhale (1995) ".strip().rsplit(' ', 1)

['Waiting to Exhale', '(1995)']

In [ ]:
movies['title'].str.rsplit(' ', n=1)

0                                [Toy Story, (1995)]
1                                  [Jumanji, (1995)]
2                         [Grumpier Old Men, (1995)]
3                        [Waiting to Exhale, (1995)]
4              [Father of the Bride Part II, (1995)]
                            ...                     
9698    [Black Butler: Book of the Atlantic, (2017)]
9699                 [No Game No Life: Zero, (2017)]
9700                                 [Flint, (2017)]
9701          [Bungo Stray Dogs: Dead Apple, (2018)]
9702          [Andrew Dice Clay: Dice Rules, (1991)]
Name: title, Length: 9703, dtype: object

In [ ]:
movies['title'].str.rsplit(' ', n=1, expand=True) # 데이터 프레임으로 처리해준다

,0,1
0,Toy Story,(1995)
1,Jumanji,(1995)
2,Grumpier Old Men,(1995)
3,Waiting to Exhale,(1995)
4,Father of the Bride Part II,(1995)
...,...,...
9698,Black Butler: Book of the Atlantic,(2017)
9699,No Game No Life: Zero,(2017)
9700,Flint,(2017)
9701,Bungo Stray Dogs: Dead Apple,(2018)


### 적용

In [ ]:
movies['title'] = movies['title'].str.strip()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9698,193581,Black Butler: Book of the Atlantic (2017),Action Animation Comedy Fantasy
9699,193583,No Game No Life: Zero (2017),Animation Comedy Fantasy
9700,193585,Flint (2017),Drama
9701,193587,Bungo Stray Dogs: Dead Apple (2018),Action Animation


In [ ]:
# k = movies.copy()

In [ ]:
# k[['title', 'year']] = movies['title'].str.rsplit(' ', n=1, expand=True) # year column 생성
# k

,movieId,title,genres,year
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,(1995)
1,2,Jumanji,Adventure Children Fantasy,(1995)
2,3,Grumpier Old Men,Comedy Romance,(1995)
3,4,Waiting to Exhale,Comedy Drama Romance,(1995)
4,5,Father of the Bride Part II,Comedy,(1995)
...,...,...,...,...
9698,193581,Black Butler: Book of the Atlantic,Action Animation Comedy Fantasy,(2017)
9699,193583,No Game No Life: Zero,Animation Comedy Fantasy,(2017)
9700,193585,Flint,Drama,(2017)
9701,193587,Bungo Stray Dogs: Dead Apple,Action Animation,(2018)


In [ ]:
movies[['title', 'year']] = movies['title'].str.rsplit(' ', n=1, expand=True) # year column 생성
movies

,movieId,title,genres,year
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,(1995)
1,2,Jumanji,Adventure Children Fantasy,(1995)
2,3,Grumpier Old Men,Comedy Romance,(1995)
3,4,Waiting to Exhale,Comedy Drama Romance,(1995)
4,5,Father of the Bride Part II,Comedy,(1995)
...,...,...,...,...
9698,193581,Black Butler: Book of the Atlantic,Action Animation Comedy Fantasy,(2017)
9699,193583,No Game No Life: Zero,Animation Comedy Fantasy,(2017)
9700,193585,Flint,Drama,(2017)
9701,193587,Bungo Stray Dogs: Dead Apple,Action Animation,(2018)


In [ ]:
movies['year'] = movies['year'].apply(lambda x: x[1:-1])
movies['year'] = movies['year'].astype(int)
movies

,movieId,title,genres,year
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995
1,2,Jumanji,Adventure Children Fantasy,1995
2,3,Grumpier Old Men,Comedy Romance,1995
3,4,Waiting to Exhale,Comedy Drama Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
9698,193581,Black Butler: Book of the Atlantic,Action Animation Comedy Fantasy,2017
9699,193583,No Game No Life: Zero,Animation Comedy Fantasy,2017
9700,193585,Flint,Drama,2017
9701,193587,Bungo Stray Dogs: Dead Apple,Action Animation,2018


In [ ]:
movies = movies[['movieId', 'title', 'year', 'genres']]
movies

,movieId,title,year,genres
0,1,Toy Story,1995,Adventure Animation Children Comedy Fantasy
1,2,Jumanji,1995,Adventure Children Fantasy
2,3,Grumpier Old Men,1995,Comedy Romance
3,4,Waiting to Exhale,1995,Comedy Drama Romance
4,5,Father of the Bride Part II,1995,Comedy
...,...,...,...,...
9698,193581,Black Butler: Book of the Atlantic,2017,Action Animation Comedy Fantasy
9699,193583,No Game No Life: Zero,2017,Animation Comedy Fantasy
9700,193585,Flint,2017,Drama
9701,193587,Bungo Stray Dogs: Dead Apple,2018,Action Animation


# 데이터 저장

In [ ]:
movies.to_csv(path + 'movies_refined.csv', index=False)
movies

,movieId,title,year,genres
0,1,Toy Story,1995,Adventure Animation Children Comedy Fantasy
1,2,Jumanji,1995,Adventure Children Fantasy
2,3,Grumpier Old Men,1995,Comedy Romance
3,4,Waiting to Exhale,1995,Comedy Drama Romance
4,5,Father of the Bride Part II,1995,Comedy
...,...,...,...,...
9698,193581,Black Butler: Book of the Atlantic,2017,Action Animation Comedy Fantasy
9699,193583,No Game No Life: Zero,2017,Animation Comedy Fantasy
9700,193585,Flint,2017,Drama
9701,193587,Bungo Stray Dogs: Dead Apple,2018,Action Animation


In [ ]:
ratings.to_csv(path + 'ratings_refined.csv', index=False)
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100784,610,166534,4.0,1493848402
100785,610,168248,5.0,1493850091
100786,610,168250,5.0,1494273047
100787,610,168252,5.0,1493846352


In [ ]:
datetime.fromtimestamp(964982703)

NameError: ignored

In [ ]:
time_unit = 2
datetime.fromtimestamp(964982703 / time_unit)

NameError: ignored

In [ ]:
from datetime import datetime
tmp = ratings.copy()
time_unit = 2
tmp['datetime'] = tmp['timestamp'].apply(lambda x: datetime.fromtimestamp(x/time_unit))
tmp

,userId,movieId,rating,timestamp,datetime
0,1,1,4.0,964982703,1985-04-16 09:22:31.500
1,1,3,4.0,964981247,1985-04-16 09:10:23.500
2,1,6,4.0,964982224,1985-04-16 09:18:32.000
3,1,47,5.0,964983815,1985-04-16 09:31:47.500
4,1,50,5.0,964982931,1985-04-16 09:24:25.500
...,...,...,...,...,...
100784,610,166534,4.0,1493848402,1993-09-01 22:56:41.000
100785,610,168248,5.0,1493850091,1993-09-01 23:10:45.500
100786,610,168250,5.0,1494273047,1993-09-04 09:55:23.500
100787,610,168252,5.0,1493846352,1993-09-01 22:39:36.000


In [156]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # TF-IDF 벡터화
# tfidf = TfidfVectorizer(stop_words='english')
# ts_movies['genres'] = ts_movies['genres'].fillna('')  # 결측값 처리
# tfidf_matrix = tfidf.fit_transform(ts_movies['genres'])

# # 코사인 유사도 계산
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# # 영화 제목과 인덱스를 매핑하는 시리즈 생성
# indices = pd.Series(ts_movies.index, index=ts_movies['title']).drop_duplicates()

# # 비슷한 영화 추천 함수
# def get_recommendations(title, cosine_sim=cosine_sim):
#     idx = indices[title]
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:11]
#     movie_indices = [i[0] for i in sim_scores]
#     return movies['title'].iloc[movie_indices]

# # for i in no_genere_title:
# #   similar_movies = get_recommendations(i)

# # print(similar_movies)


In [ ]:
# # 비슷한 영화 추천 후, 해당 영화의 장르를 추가하는 함수
# def similar_title(movie_idx):
#     # 주어진 영화 인덱스에 해당하는 영화 제목
#     movie_title = ts_movies.loc[movie_idx, 'title']

#     # 'movie_title' 영화의 유사한 영화 가져오기
#     similar_movies = content_recommender(movie_title, 1)

#     print(f"영화 '{movie_title}'와 유사한 영화 인덱스:")
#     # print(similar_movies)

#     similar_movie_indices = []  # 유사한 영화의 인덱스를 저장할 리스트
#     similar_movie_genres  = []
#     # similar_movies의 인덱스 번호와 해당 영화 제목 출력
#     for idx, similar_movie_title in enumerate(similar_movies):
#         similar_movie_idx = ts_movies[ts_movies['title'] == similar_movie_title].index
#         if len(similar_movie_idx) > 0:
#             similar_movie_indices.append(similar_movie_idx[0])

#     for similar_movie_idx in similar_movie_indices:
#         similar_movie_genre = ts_movies.loc[similar_movie_idx, 'genres']
#         similar_movie_genres.append(similar_movie_genre)

#     return similar_movie_indices, similar_movie_genres

# def update_movie_genres(movie_indices, similar_movie_genres):
#     for i, movie_idx in enumerate(movie_indices):
#         ts_movies.at[movie_idx, 'genres'] = similar_movie_genres

# update_movie = ts_movies.copy()

# # movie_indices 리스트의 영화들에 대해 장르 추가
# for movie_idx in (movie_indices):
#     similar_movie_indices, similar_movie_genres = similar_title(movie_idx)
#     update_movie = update_movie_genres(movie_indices, similar_movie_genres)
#     print(similar_movie_indices, similar_movie_genres)
#     # print(similar_movie_genres)
